In [3]:
import numpy
from time import time

def string_to_float(ret):
    
    nbyte = len(ret)/8
    float_type = None
    if nbyte == 2:
        float_type = numpy.float16
    if nbyte == 4:
        float_type = numpy.float32
    if nbyte == 8:
        float_type = numpy.float64
    
    # parse ret by bytes
    list_str = []
    while(len(list_str) < nbyte): 
        list_str.append(ret[:8])
        ret = ret[8:]
        
    # convert string to int
    list_bytes = []
    for str_bytes in list_str:
        list_bytes = [int(str_bytes, 2)] + list_bytes # reverse order to compensate endianness
        
    ret = numpy.frombuffer(bytes(list_bytes), dtype=float_type)
    return ret[0]

def float_to_string(f):

    byte_f = f.tobytes()
    nbyte = len(byte_f)
    
    ret = ""
    for i in range(0, nbyte):
        str_bytes = bin(byte_f[i])[2:] # convert each byte to string
        while(len(str_bytes) < 8):
            str_bytes = "0" + str_bytes # pad with 0s on MSB side
        ret = str_bytes + ret # concat all
    return ret

In [4]:
def show_integer_properties():
    Table = "{:<6} {:<22} {:<22} {:<22}"
    print(Table.format("Bytes","Largest Unsigned Int","Minimum Signed Int","Maximum Signed Int"))
    
    for ii in range(1, 9):
        int_largest_unsigned_int = 2**(8*ii)-1
        int_minimum_signed_int = -(int_largest_unsigned_int+1 >> 1)
        int_maximum_signed_int = (int_largest_unsigned_int-1) >> 1
        print(Table.format(str(ii), str(int_largest_unsigned_int), str(int_minimum_signed_int), str(int_maximum_signed_int)))

In [5]:
show_integer_properties()

Bytes  Largest Unsigned Int   Minimum Signed Int     Maximum Signed Int    
1      255                    -128                   127                   
2      65535                  -32768                 32767                 
3      16777215               -8388608               8388607               
4      4294967295             -2147483648            2147483647            
5      1099511627775          -549755813888          549755813887          
6      281474976710655        -140737488355328       140737488355327       
7      72057594037927935      -36028797018963968     36028797018963967     
8      18446744073709551615   -9223372036854775808   9223372036854775807   


In [6]:
def estimate_wrap_around():
    
    m = numpy.array([1], dtype=numpy.int16)
    seconds_start = time()
    while m[0] > 0:
        m[0] += 1
    seconds_runtime = time() - seconds_start
    str_line1 = "measured 16-bit time (microseconds): " + str(seconds_runtime * 1e6)
    
    str_line0 = "estimated 8-bit time (nanoseconds): " + str(seconds_runtime * 2**(8-16) * 1e9)
    str_line2 = "estimated 32-bit time (seconds): " + str(seconds_runtime * 2**(32-16))
    str_line3 = "estimated 64-bit time (years): " + str(seconds_runtime * 2**(64-16) /60/60/365)
    
    print(str_line0)
    print(str_line1)
    print(str_line2)
    print(str_line3)

In [7]:
estimate_wrap_around()

estimated 8-bit time (nanoseconds): 256807.54333734512
measured 16-bit time (microseconds): 65742.73109436035
estimated 32-bit time (seconds): 4308.515625
estimated 64-bit time (years): 14082902.362009132


In [ ]:
# if type(f) == numpy.float16:
#     fbits = 10
#     ebias = -15
#     nbyte = 2
#     ebits = 5

# if type(f) == numpy.float32:
#     fbits = 23
#     ebias = -127
#     nbyte = 4
#     ebits = 8

# if type(f) == numpy.float64:
#     fbits = 52
#     ebias = -1023
#     nbyte = 8
#     ebits = 11

# if type(f) == numpy.float128:
#     fbits = 112
#     ebias = -16383
#     nbyte = 16
#     ebits = 15

In [11]:
def maximum_consecutive_int_value(arg):
    fbits = None
    ebias = None
    nbyte = None
    
    if arg == numpy.float16:
        fbits = 10
        ebias = -15
        nbyte = 2
        
    if arg == numpy.float32:
        fbits = 23
        ebias = -127
        nbyte = 4
        
    if arg == numpy.float64:
        fbits = 52
        ebias = -1023
        nbyte = 8
        
    ret = bin(fbits-ebias+1)[2:] # compute exponent into string
    ret = "0" + ret + "0"*fbits # "bitshift" exponent to right place
    
    return string_to_float(ret)

In [12]:
maximum_consecutive_int_value(numpy.float64)

9007199254740992.0

In [53]:
def largest_double():
    
    fbits = 52
    ebits = 11
    
    ret = bin((2**ebits)-2)[2:] # compute exponent into string
    ret = "0" + ret + "1"*fbits # "bitshift" exponent to right place and pad with 1s
    
    return string_to_float(ret)

In [54]:
largest_double()

1.7976931348623157e+308

In [57]:
def smallest_double():
    
    nbyte = 8
    
    ret = "0"*(nbyte*8-1) + "1" # all 0s except the LSB
    
    return string_to_float(ret)    

In [58]:
smallest_double()

5e-324

In [49]:
def largest_single():
    
    fbits = 23
    ebits = 8
    
    ret = bin((2**ebits)-2)[2:] # compute exponent into string
    ret = "0" + ret + "1"*fbits # "bitshift" exponent to right place and pad with 1s
    
    return string_to_float(ret)

In [50]:
largest_single()

3.4028235e+38

In [59]:
def smallest_single():
    
    nbyte = 4
    
    ret = "0"*(nbyte*8-1) + "1" # all 0s except the LSB
    
    return string_to_float(ret)

In [60]:
smallest_single()

1e-45

In [62]:
def breakdown_float(f, /):
    
    ret = {"sign": None, "fraction": None, "exponent": None, "subnormal": None}
    fbits = None
    ebits = None
    
    if type(f) == numpy.float16:
        fbits = 10
        ebits = 5
    
    if type(f) == numpy.float32:
        fbits = 23
        ebits = 8
    
    if type(f) == numpy.float64:
        fbits = 52
        ebits = 11

    str_f = float_to_string(f)
    
    ret["sign"] = int(str_f[0], 2)
    ret["exponent"] = int(str_f[1:1+ebits], 2)
    ret["fraction"] = int(str_f[2+ebits:2+ebits+fbits], 2)
    ret["subnormal"] = ret["exponent"] == 0
    
    return ret

In [63]:
m = numpy.float64(2.5)
d = breakdown_float(m)
print(d)

{'sign': 0, 'fraction': 1125899906842624, 'exponent': 1024, 'subnormal': False}


In [64]:
def construct_float(float_parms, /, *, float_type=float, subnormal=False):
    
    fbits = None
    ebits = None
    
    if float_type == numpy.float16:
        fbits = 10
        ebits = 5
    
    if float_type == numpy.float32 or float_type == float:
        fbits = 23
        ebits = 8
    
    if float_type == numpy.float64:
        fbits = 52
        ebits = 11
    
    sign = float_parms["sign"]
    if subnormal:
        exponent = 0
    else:
        exponent = float_parms["exponent"]
    fraction = float_parms["fraction"]
    
    sign = str(sign)
    exponent = bin(exponent)[2:]
    fraction = bin(fraction)[2:]
    
    if(len(exponent) > ebits or len(fraction) > fbits):
        raise ValueError("inconstructible float!")
    
    while(len(exponent) < ebits):
        exponent = "0" + exponent
    while(len(fraction) < fbits):
        fraction = "0" + fraction
        
    str_f = sign+exponent+fraction
    
    return string_to_float(str_f)

In [73]:
f = {'sign': 0, 'fraction': 1125899906842624, 'exponent': 1024, 'subnormal': True}
g = construct_float(f, float_type=numpy.float64, subnormal=True)
h = type(g)
f, g, h

({'sign': 0,
  'fraction': 1125899906842624,
  'exponent': 1024,
  'subnormal': True},
 5.562684646268003e-309,
 numpy.float64)

In [16]:
def get_next_float(start_float, /, *, index=1):
    
    restore_type = False
    float_type = type(start_float)
    
    if(float_type == float):
        start_float = numpy.float32(start_float)
        restore_type = True
    elif(float_type != numpy.float16 and
        float_type != numpy.float32 and
        float_type != numpy.float64):
        raise TypeError("wrong start_float type!", float_type)
        
    if(index <= 0):
        raise ValueError("non-positive index!", index)
       
    index_type = type(index)
    if(index_type != int and
      index_type != numpy.intx):
        raise TypeError("wrong index type!", index_type)
    
    str_ret = float_to_string(start_float)
    list_ret = list(str_ret)
    
    while(index > 0):
        for i in range(len(list_ret)-1, -1, -1):
            if list_ret[i] == "0":
                list_ret[i] = "1"
                index -= 1
                break
            else:
                list_ret[i] = "0"
            
    str_ret = "".join(list_ret)
    ret = string_to_float(str_ret)
    
    if restore_type:
        return float(ret)
    else:
        return ret

In [15]:
get_next_float(2.6, index=0)

ValueError: ('non-positive index!', 0)

In [175]:
a = "1234"
for i in range(len(a)-1, -1, -1):
    print(a[i])
a[3] = "5"

4
3
2
1


TypeError: 'str' object does not support item assignment

In [43]:
bin(bytes(numpy.float16(2048))[1])

'0b1101000'

In [68]:
a = numpy.float16(2048).tobytes()
bin(a[1])

'0b1101000'

In [260]:
numpy.frombuffer(bytes([int(a[1], 2), int(a[0], 2)]), dtype=numpy.float16)

array([2048.], dtype=float16)

In [212]:
numpy.float16(10) * 2

20.0

In [78]:
b = maximum_consecutive_int_value(numpy.float32)
c = numpy.frombuffer(b, dtype=numpy.float32)
b, c

(b'\x00\x00\x80K', array([16777216.], dtype=float32))

In [111]:
bytes([1,255])

b'\x01\xff'

In [134]:
a = int("ffff", 16)
b = bin(a)
c = bytes(c)[::-1]
d = numpy.frombuffer(c, dtype=numpy.float32)
a, b, c, d

(65535,
 '0b1111111111111111',
 b'\x00\x00\x80K',
 array([16777216.], dtype=float32))

In [210]:
a = int("0110100000000000", 2)
b = hex(a & 0xff)
c = hex(a >> 8)
d = bytes([int(b, 16), int(c, 16)])
e = numpy.frombuffer(d, dtype=numpy.float16)
a, b, c, d, e[0]

(26624, '0x0', '0x68', b'\x00h', 2048.0)

In [159]:
b = bytes([int("0xff", 16), int("0x7b", 16)])
c = numpy.frombuffer(b, dtype=numpy.float16)
b, c

(b'\xff{', array([65504.], dtype=float16))

In [54]:
import numpy
# 0100_0000_0000_0000
# 4000
b = b'\x00\x40'
c = numpy.frombuffer(b, numpy.float16)
b, c

(b'\x00@', array([2.], dtype=float16))

In [ ]:
import numpy
# 0100_0110_0000_0000
# 
b = b'\x00\x40'
c = numpy.frombuffer(b, numpy.float16)
b, c

In [92]:
674 / 256 * 1000

2632.8125

In [98]:
import numpy as np
from time import time

def estimate_wrap_around():
    m = np.array([1], dtype=np.int8)
    start = time()
    while (m[0] << 0) > 0:
        m[0] += 1
    print("estimated 8-bit time (nanoseconds):", (time()-start)*1e9)
    
    m = np.array([1], dtype=np.int16)
    start = time()
    while (m[0] << 0) > 0:
        m[0] += 1
    print("measured 16-bit time (microseconds):", (time()-start)*1e6)
    
estimate_wrap_around()

def estimate_wrap_around():
    m = np.array([1], dtype=np.int8)
    start = time()
    while (m[0] << 1) > 0:
        m[0] += 1
    print("estimated 8-bit time (nanoseconds):", (time()-start)*2e9)
    
    m = np.array([1], dtype=np.int16)
    start = time()
    while (m[0] << 1) > 0:
        m[0] += 1
    print("measured 16-bit time (microseconds):", (time()-start)*2e6)
    
estimate_wrap_around()

estimated 8-bit time (nanoseconds): 1000165.9393310547
measured 16-bit time (microseconds): 200034.85679626465
estimated 8-bit time (nanoseconds): 1980304.7180175781
measured 16-bit time (microseconds): 663982.8681945801


In [35]:
digits = []
for i in range(0, 11):
    digits.append(1/(2**i))
digits

[1.0,
 0.5,
 0.25,
 0.125,
 0.0625,
 0.03125,
 0.015625,
 0.0078125,
 0.00390625,
 0.001953125,
 0.0009765625]

In [72]:
def is_precise_float16(arg):
    
    num_fraction_bits = 10
    e_max = 15
    
    digits = []
    for i in range(0, num_fraction_bits+1):
        digits.append(1/(2**i))
    
    exponent = 0
    while(arg > 1):
        arg /= 2
        exponent += 1
#     print("exponent is", exponent)
    if exponent > e_max:
        return False
    
    fraction = 0
    for digit in digits:
        if arg >= digit:
#             print(digit)
            arg -= digit
            fraction += digit
#     print("fraction is", fraction)
            
#     print(fraction * (2**exponent))
            
    return (arg == 0)

In [77]:
test_num = 1
result = True
while(result == True):
#     print("test_num is", test_num)
    result = is_precise_float16(test_num)
    test_num += 1
#     print("")
test_num-1

1025

In [78]:
def is_precise_float32(arg):
    
    num_fraction_bits = 23
    e_max = 127
    
    digits = []
    for i in range(0, num_fraction_bits+1):
        digits.append(1/(2**i))
    
    exponent = 0
    while(arg > 1):
        arg /= 2
        exponent += 1
#     print("exponent is", exponent)
    if exponent > e_max:
        return False
    
    fraction = 0
    for digit in digits:
        if arg >= digit:
#             print(digit)
            arg -= digit
            fraction += digit
#     print("fraction is", fraction)
            
#     print(fraction * (2**exponent))
            
    return (arg == 0)

In [92]:
is_precise_float32(2**23+1)

False

In [80]:
test_num = 1
result = True
while(result == True):
#     print("test_num is", test_num)
    result = is_precise_float32(test_num)
    test_num += 1
#     print("")
test_num-1

KeyboardInterrupt: 

In [32]:
barray = b'\x01\x02\x03'
b_little = int.from_bytes(barray, byteorder="little")
b_big = int.from_bytes(barray, byteorder="big")

print(b_little)
print(b_big)

0x010203, 0x030201

197121
66051


(66051, 197121)

In [ ]:
# 2.5 is 2097152 fraction and 128 exponent
# binary to float 32 representation
x = 0b0100_0000_0010_0000_0000_0000_0000_0000

print(x.to_bytes(length=4, ))

In [118]:
0b0111_1011_1111_1111
byte_array = b'\x07\x0b\x0f\x0f'
int.from_bytes(byte_array, byteorder="little")

252644103

In [50]:
# a = numpy.float16(67)
# b = a.tobytes()
b = b'\x07\x0b\x0f\x0f'
c = np.frombuffer(b, dtype=np.float16)
b, c

(b'\x07\x0b\x0f\x0f', array([0.0002145, 0.0004308], dtype=float16))

In [7]:
a = np.float16(2048)
b = a.tobytes()
c = np.frombuffer(b, dtype=np.float16)
a, b, c

(2048.0, b'\x00h', array([2048.], dtype=float16))

In [10]:
help(bytes)

Help on class bytes in module builtins:

class bytes(object)
 |  bytes(iterable_of_ints) -> bytes
 |  bytes(string, encoding[, errors]) -> bytes
 |  bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
 |  bytes(int) -> bytes object of size given by the parameter initialized with null bytes
 |  bytes() -> empty bytes object
 |  
 |  Construct an immutable array of bytes from:
 |    - an iterable yielding integers in range(256)
 |    - a text string encoded using the specified encoding
 |    - any object implementing the buffer API.
 |    - an integer
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__

In [42]:
b = b'\x00\x80'
c = np.frombuffer(b, dtype=np.float16)
b, c

(b'\x00\x80', array([-0.], dtype=float16))

In [47]:
b = b'\x7b\xff'
b[::-1]
c = np.frombuffer(b, dtype=np.float16)
b, c

(b'{\xff', array([nan], dtype=float16))

In [ ]:
def binary_to_float16(arg):
    
    sign = (arg & 0b1000_0000_0000_0000) >> 15
    expo = (arg & 0b0111_1100_0000_0000) >> 10
    frac = (arg & 0b0000_0011_1111_1111)
    
    real_frac = 0;
    for i in range(10):
        real_frac += (frac & 0b1)
        real_frac /= 2
        frac = frac >> 1
    
    return (sign*-2+1) * (2**(expo-15)) * (1+real_frac)